In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Low order ODE integrators ##

Sometimes it is better to use a low order ODE integrator and just hammer the number of steps, rather than a higher order integrator that requires fewer function calls.  The "workhorse" integrator for ODE's is probably the [fourth order Runge-Kutta method](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods), colloquially known as [RK4](https://en.wikipedia.org/wiki/List_of_Runge%E2%80%93Kutta_methods).  The lowest order method is Euler's method, which is normally introduced primarily to show how bad it is!  Here let's look at two methods which are second order and involve two derivative evaluations.

We are interested in the solution of the ODE
$$
\frac{d\vec{y}}{dt} = f(t,\vec{y})
$$
given (well posed) initial values $\vec{y}(t_0)$.  We imagine taking steps, of size $h$, in $t$ and updating $\vec{y}$ through a sequence of values $\vec{y}_0$, $\vec{y}_1$, ..., etc.

### Midpoint method ###

Perhaps the most obvious second order method (related to the "leap frog" method for symplectic systems) is the [midpoint method](https://en.wikipedia.org/wiki/Midpoint_method).  Here we use Euler's method to get an estimate for $\vec{y}$ at the midpoint of the interval, then evaluate $f$ there and use that value to take the step.
$$
  y_{n+1} = y_n + h\,f\left(t_n+\frac{h}{2},y_n+\frac{h}{2}f(t_n,y_n)\right)
$$
We can write this in a way slightly more reminiscent of the Runge-Kutta family of integrators by saying
$$
  k_1 = f(t_n,y_n) \quad , \quad
  y_{n+1} = y_n + h\,f\left(t_n+\frac{1}{2}h,y_n+\frac{1}{2}h\,k_1\right)
$$

### Heun's method ###

A lesser known method, but one which feels very "obvious" in hindsight, is [Heun's method](https://en.wikipedia.org/wiki/Heun's_method#Runge.E2.80.93Kutta_method).  In this method we make an estimate of the slope at the beginning and end of the interval, average them and then take a step using that slope.  Specifically (in "Runge-Kutta" language):
$$
  k_1 = f(t_n,y_n) \quad , \quad
  k_2 = f(t_n+h,y_n+hk_1) \quad , \quad
  y_{n+1} = y_n + \frac{h}{2}\left(k_1+k_2\right)
$$

### Test case ###

Let's consider a simple numerical test.  We evolve
$$
  \frac{d}{dt}\left(\begin{array}{c} y_1 \\ y_2\end{array}\right) =
  \left(\begin{array}{c} -y_2 \\ y_1\end{array}\right)
$$
starting from $\vec{y}(0)=[1,0]$.  This should give $\vec{y}(t)=[\cos(t),\sin(t)]$, or a point in the 2D plane undergoing harmonic/circular motion.  We want to see how our integrators perform as a function of step size.

The period of the orbit is $2\pi$, and this will set the characteristic timescale.  We don't need to evolve the whole way, but we don't want to evolve to some super-special point like $\pi/2$ or something either as this could be misleading.  Instead let's evolve to $t=1$.

In [2]:
def f(t,y):
    """The derivative function -- note this doesn't actually depend on t."""
    return(np.array([-y[1],y[0]]))

In [3]:
def midpoint_step(t,y,h,f):
    """Takes one midpoint step, of size h."""
    h2 = 0.5*h
    k1 = f(t,y)
    dy = h*f(t+h2,y+h2*k1)
    return(y+dy)

In [4]:
def heun_step(t,y,h,f):
    """Takes one Heun step, of size h."""
    k1 = f(t,y)
    k2 = f(t+h,y+h*k1)
    dy = 0.5*h*(k1+k2)
    return(y+dy)

In [5]:
t0,t1 = 0.0,1.0
tt,y0 = t0,np.array([1.0,0.0])
yexact= y0.copy()
ymidp = y0.copy()
yheun = y0.copy()
Nstep = 8
hh    = (t1-t0)/Nstep
#
for i in range(Nstep):
    yexact = np.array([np.cos(tt+hh),np.sin(tt+hh)])
    ymidp  = midpoint_step(tt,ymidp,hh,f)
    yheun  = heun_step(tt,yheun,hh,f)
    tt     = tt + hh
#
print("t_final=",tt)
print("y_exact=",yexact)
print("y_midp =",ymidp)
print("y_heun =",yheun)

t_final= 1.0
y_exact= [0.54030231 0.84147098]
y_midp = [0.53825088 0.84307436]
y_heun = [0.53825088 0.84307436]


So both did pretty well, in fact they were essentially identical!  Why do you think this would be?  Let's look at convergence as a function of the number of steps.

In [6]:
for istep in range(1,5):
    t0,t1 = 0.0,1.0
    tt,y0 = t0,np.array([1.0,0.0])
    yexact= y0.copy()
    ymidp = y0.copy()
    yheun = y0.copy()
    Nstep = 2**istep
    hh    = (t1-t0)/Nstep
    for i in range(Nstep):
        yexact = np.array([np.cos(tt+hh),np.sin(tt+hh)])
        ymidp  = midpoint_step(tt,ymidp,hh,f)
        yheun  = heun_step(tt,yheun,hh,f)
        tt     = tt + hh
    print(str(Nstep).rjust(3)," ",(ymidp-yexact)/yexact,\
                              " ",(yheun-yexact)/yexact)

  2   [-0.04567315  0.03984572]   [-0.04567315  0.03984572]
  4   [-0.01404123  0.00847481]   [-0.01404123  0.00847481]
  8   [-0.00379681  0.00190544]   [-0.00379681  0.00190544]
 16   [-0.00098247  0.00044786]   [-0.00098247  0.00044786]


Note that the error should be going down as $h^2$, and it roughly is.  Doubling the number of steps halves the interval so the error should be about $4\times$ as small each time.  Which is pretty close to right!

What about if we change the problem?  Rather than an oscillating exponential, what if we go to something non-linear?  Let's try
$$
  \frac{dy}{dt} = y^2 \quad , \quad y(0) = 1
$$
with solution $y(t) = (1-t)^{-1}$.

In [7]:
def f2(t,y):
    """The derivative function -- note this doesn't actually depend on t
    and doesn't really need to be an array."""
    return(np.array([y[0]**2]))

In [8]:
for istep in range(2,6):
    t0,t1 = 0.0,0.5
    tt,y0 = t0,np.array([1.0])
    yexact= y0.copy()
    ymidp = y0.copy()
    yheun = y0.copy()
    Nstep = 2**istep
    hh    = (t1-t0)/Nstep
    for i in range(Nstep):
        yexact = np.array([1/(1-(tt+hh))])
        ymidp  = midpoint_step(tt,ymidp,hh,f2)
        yheun  = heun_step(tt,yheun,hh,f2)
        tt     = tt + hh
    print(str(Nstep).rjust(3)," ",(ymidp-yexact)/yexact," ",\
                                  (yheun-yexact)/yexact)

  4   [-0.01672898]   [-0.01239795]
  8   [-0.00496772]   [-0.00350953]
 16   [-0.00135122]   [-0.00092843]
 32   [-0.00035193]   [-0.00023826]


So in this case the Heun method performs slightly better than midpoint, though both are doing pretty well.  While there is no guarantee that Heun's method outperforms midpoint, it has been my experience that it often does.  So if you want a nice, low-order integrator then it's not a bad choice!